In [1]:
import math
import pandas as pd
from datetime import datetime
pd.set_option('display.float_format',lambda x : '%.3f' % x)

In [2]:
import QUANTAXIS as QA
from PaperTrader import Papertest,Paperpositon,ORDER_DIRECTION,MARKET

jqdatasdk not installed


D:\APP\anaconda3\envs\quantaxis\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
Account =Papertest()
Account

In [4]:
data_forbacktest = pd.read_csv("./data/未复权数据回测样本.csv")
data_forbacktest.date = data_forbacktest.date.apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d"))
data_forbacktest

,date,code,open,high,low,close,volume,duo,kong
0,2009-10-16,601888,16.660,16.790,15.370,15.410,970132.000,False,True
1,2009-10-19,601888,15.810,16.120,15.400,15.840,613624.000,True,False
2,2009-10-20,601888,15.650,17.420,15.620,17.420,547075.000,False,False
3,2009-10-21,601888,17.560,17.700,16.950,17.060,600277.000,False,False
4,2009-10-22,601888,16.890,17.450,16.650,17.260,400865.000,False,False
...,...,...,...,...,...,...,...,...,...
2708,2021-03-15,601888,303.860,309.000,290.500,297.800,125917.000,False,False
2709,2021-03-16,601888,299.000,301.880,285.620,296.570,105408.000,False,False
2710,2021-03-17,601888,292.950,309.790,287.110,308.070,105216.000,False,False
2711,2021-03-18,601888,313.000,320.250,308.800,312.700,116849.000,False,False


In [5]:
split_devi = pd.read_csv("./data/除权除息.csv")
split_devi.index= split_devi.datetime.apply(lambda x: datetime.strptime(x,"%Y-%m-%d").date())
split_devi

,code,split,datetime,dividend
datetime,,,,
2017-06-19,601888,1.000,2017-06-19,0.000
2011-06-28,601888,0.000,2011-06-28,0.100
2012-06-12,601888,0.000,2012-06-12,0.100
2013-05-28,601888,0.000,2013-05-28,0.350
2014-06-24,601888,0.000,2014-06-24,0.400
2015-07-14,601888,0.000,2015-07-14,0.460
2016-06-03,601888,0.000,2016-06-03,0.500
2017-06-16,601888,0.000,2017-06-16,1.000
2018-06-22,601888,0.000,2018-06-22,0.520


In [6]:
def get_dividend(divi_date:datetime):
    if divi_date.date() in split_devi.index:
        divi_df = split_devi.loc[divi_date.date()]
        if isinstance(divi_df,pd.Series):
            return [divi_df.to_dict()]
        elif isinstance(divi_df,pd.DataFrame):
            return divi_df.to_dict(orient="records")

get_dividend(datetime(2017,6,19))

[{'code': 601888, 'split': 1.0, 'datetime': '2017-06-19', 'dividend': 0.0}]

In [7]:
for itemdx, item in data_forbacktest.iterrows():
    Account.on_current_time(item.date)
    Account.on_price_change(item.code, item.close)
    Account.on_dividend(get_dividend(item.date))
    if item.code not in Account.position.keys():
        Account.position[item.code] = Paperpositon(code=item.code,
                                                   t=Account.t,
                                                   code_type=MARKET.stock_cn)

    
    if item.duo:
        if Account.position[item.code].gpye == 0:
            order_buy = Account.send_order(
                code=item.code,
                order_time=item.date,
                order_volume=math.ceil(
                    Account.cash_available / item.close / 100 /
                    (1 + Account.commisson)) * 100 - 100,
                order_price=item.close,
                order_type=ORDER_DIRECTION.BUY)
            Account.make_deal(order_buy)
            print(f"{item.date},做多:{order_buy}")
#             print(Account.position[item.code])

    elif item.kong:
        if Account.position[item.code].kyye > 0:
#             print(Account.position[item.code])
            order_sell = Account.send_order(
                code=item.code,
                order_time=item.date,
                order_volume=Account.position[item.code].kyye,
                order_price=item.close,
                order_type=ORDER_DIRECTION.SELL)
            Account.make_deal(order_sell)
            print(f"{item.date},做空:{order_sell}")
    elif Account.position[item.code].dividend_money > 0:
        # 分红继续购买，是否足够购买，单股票时候可以按照再以开盘价购入的形式使用
        fengong = math.ceil(Account.cash_available / item.open / 100 /(1 + Account.commisson)) * 100 - 100
        if fengong>0:
            order_divi = Account.send_order(
                code=item.code,
                order_time=item.date,
                order_volume=math.ceil(Account.cash_available / item.open / 100 /
                                       (1 + Account.commisson)) * 100 - 100,
                order_price=item.close,
                order_type=ORDER_DIRECTION.BUY)
            Account.make_deal(order_divi)
            print(f"{item.date},分红继续购买:{order_divi}")

    Account.settle()

2009-10-19 00:00:00,做多:39a543f3-9619-11eb-abf6-309c23e8c748
2009-11-02 00:00:00,做空:39a85125-9619-11eb-a88a-309c23e8c748
2009-11-03 00:00:00,做多:39a9422a-9619-11eb-b4d1-309c23e8c748
2010-05-20 00:00:00,做空:39c31020-9619-11eb-b4b7-309c23e8c748
2010-05-24 00:00:00,做多:39c41710-9619-11eb-bd21-309c23e8c748
2011-01-06 00:00:00,做空:39e09adf-9619-11eb-88a3-309c23e8c748
分红送股： {'code': 601888, 'split': 0.0, 'datetime': '2011-06-28', 'dividend': 0.1}
2011-10-18 00:00:00,做多:39fda759-9619-11eb-9d72-309c23e8c748
2012-01-06 00:00:00,做空:3a094b8b-9619-11eb-a7c6-309c23e8c748
2012-01-09 00:00:00,做多:3a0a70be-9619-11eb-8544-309c23e8c748
2012-01-16 00:00:00,做空:3a0c459a-9619-11eb-8ec9-309c23e8c748
2012-01-17 00:00:00,做多:3a0d72f4-9619-11eb-936b-309c23e8c748
2012-05-25 00:00:00,做空:3a1cc7f8-9619-11eb-bc53-309c23e8c748
2012-05-28 00:00:00,做多:3a1db261-9619-11eb-8eac-309c23e8c748
分红送股： {'code': 601888, 'split': 0.0, 'datetime': '2012-06-12', 'dividend': 0.1}
2012-09-17 00:00:00,做空:3a2d559a-9619-11eb-ba26-309c23e8c748


In [8]:
Account.order_hisotry_dataframe

,datetime,order_id,order_type,price,volume,is_frozen,commission,tax,money
0,2009-10-19,39a543f3-9619-11eb-abf6-309c23e8c748,1,15.840,6300,0,9.979,0,99801.979
1,2009-11-02,39a85125-9619-11eb-a88a-309c23e8c748,-1,15.690,-6300,0,9.885,98.847,-98738.268
2,2009-11-03,39a9422a-9619-11eb-b4d1-309c23e8c748,1,16.010,6100,0,9.766,0,97670.766
3,2010-05-20,39c31020-9619-11eb-b4b7-309c23e8c748,-1,16.790,-6100,0,10.242,102.419,-102306.339
4,2010-05-24,39c41710-9619-11eb-bd21-309c23e8c748,1,17.740,5800,0,10.289,0,102902.289
...,...,...,...,...,...,...,...,...,...
56,2020-11-12,3b80db37-9619-11eb-a6a2-309c23e8c748,1,196.150,1700,0,33.346,0,333488.345
57,2020-11-16,3b821293-9619-11eb-9231-309c23e8c748,-1,186.450,-1700.0,0,31.697,316.965,-316616.339
58,2020-11-23,3b83c13e-9619-11eb-95f8-309c23e8c748,1,187.890,1700,0,31.941,0,319444.941
59,2020-12-09,3b86c1d4-9619-11eb-ad0a-309c23e8c748,-1,202.330,-1700.0,0,34.396,343.961,-343582.643


In [9]:
data = pd.io.json.json_normalize(Account.settle_history)
data[data.position.apply(len)>0]

<ipython-input-9-429f9d0be30c>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = pd.io.json.json_normalize(Account.settle_history)


,datetime,all_money,positon_money,frozen_money,cash_available,all_float_profit,position
1,2009-10-19,99990.021,99792.000,0.000,198.021,-9.979,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
2,2009-10-20,109944.021,109746.000,0.000,198.021,9944.021,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
3,2009-10-21,107676.021,107478.000,0.000,198.021,7676.021,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
4,2009-10-22,108936.021,108738.000,0.000,198.021,8936.021,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
5,2009-10-23,108117.021,107919.000,0.000,198.021,8117.021,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
...,...,...,...,...,...,...,...
2708,2021-03-15,489453.872,476480.000,0.000,12973.872,143966.752,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
2709,2021-03-16,487485.872,474512.000,0.000,12973.872,141998.752,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
2710,2021-03-17,505885.872,492912.000,0.000,12973.872,160398.752,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
2711,2021-03-18,513293.872,500320.000,0.000,12973.872,167806.752,"[{'code': 601888, 'code_type': 'stock_cn', 'co..."
